### charger les docs csv



In [42]:
import sqlite3
import pandas as pd
import os

data_folder = './eleves/'
csv_folder = os.path.join(data_folder, 'csv')
sql_folder = os.path.join(data_folder, 'SQL')

if not os.path.exists(sql_folder):
    os.makedirs(sql_folder)

csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

for file in csv_files:
    filename = os.path.join(csv_folder, file)
    sqlite_filename = os.path.join(sql_folder, file[:-3] + "sqlite3")

    con = sqlite3.connect(sqlite_filename)

    if file == "Sondage_LV2.csv":
        pd.read_csv(filename, encoding='latin1', usecols=['Nom','mail','Langues']).to_sql(file[:-4], con, if_exists='replace', index=False)
    else:
        pd.read_csv(filename, encoding='latin1', usecols=['Nom','Mail','Note/10']).to_sql(file[:-4], con, if_exists='replace', index=False)

    con.close()


### charger les docs json

je ne sais plus ce que fait le premier mais il faut utiliser le deuxième

In [54]:
import sqlite3
import pandas as pd
import os
import json

data_folder = './eleves/'
json_folder = os.path.join(data_folder, 'json')
sql_folder = os.path.join(data_folder, 'SQL')

if not os.path.exists(sql_folder):
    os.makedirs(sql_folder)

json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

for file in json_files:
    filename = os.path.join(json_folder, file)
    sqlite_filename = os.path.join(sql_folder, file[:-4] + "sqlite3")
    con = sqlite3.connect(sqlite_filename)
    with open(filename, 'r', encoding='utf-8-sig') as json_file:
        data = json.load(json_file)
        if file == "Sondage_LV2.json":
            pd.DataFrame(data, columns=['Nom','Prénom','mail','Langues']).to_sql(file[:-5], con, if_exists='replace', index=False)
        else:
            pd.DataFrame(data, columns=['Nom','Prénom','mail','Note/10']).to_sql(file[:-5], con, if_exists='replace', index=False)

    con.close()


In [143]:
import sqlite3
import pandas as pd
import os
import json
import numpy as np

data_folder = './eleves/'
json_folder = os.path.join(data_folder, 'json')
sql_folder = os.path.join(data_folder, 'SQL')

if not os.path.exists(sql_folder):
    os.makedirs(sql_folder)

db_path = os.path.join(sql_folder, 'data.sqlite3')
conn = sqlite3.connect(db_path)

json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

for file in json_files:
    table_name = os.path.splitext(file)[0]
    json_file_path = os.path.join(json_folder, file)

    with open(json_file_path, 'r', encoding='utf-8-sig') as json_file:
        data = json.load(json_file)
        
        if file == "Sondage_LV2.json":
            df = pd.DataFrame(data, columns=['Nom','Prénom','mail','Langues'])
        else:
            df = pd.DataFrame(data, columns=['Nom','Prénom','mail','Note/10'])
            df['Note/10'] = df['Note/10'].replace('', np.nan)
            df['Note/10'] = df['Note/10'].astype(float)

        df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()


### charger les docs xlsx

In [2]:
import sqlite3
import pandas as pd
import os

data_folder = './eleves/'
xlsx_folder = os.path.join(data_folder, 'xlsx')
sql_folder = os.path.join(data_folder, 'SQL')

if not os.path.exists(sql_folder):
    os.makedirs(sql_folder)

xlsx_files = [f for f in os.listdir(xlsx_folder) if f.endswith('.xlsx')]

for file in xlsx_files:
    filename = os.path.join(xlsx_folder, file)
    sqlite_filename = os.path.join(sql_folder, file[:-4] + "sqlite3")

    con = sqlite3.connect(sqlite_filename)

    if file == "Sondage_LV2.xlsx":
        pd.read_excel(filename, usecols=['Nom','Prénom','mail','Langues']).to_sql(file[:-5], con, if_exists='replace', index=False)
    else:
        pd.read_excel(filename, usecols=['Nom','Prénom','Mail','Note/10']).to_sql(file[:-5], con, if_exists='replace', index=False)

    con.close()


### Find table names

entry: chemin de la database

In [134]:
import sqlite3

def list_tables(filename):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    conn.close()
    return [table[0] for table in tables]

Data = 'eleves/SQL/data.sqlite3'
print("Tables in the database:", list_tables(Data))

Tables in the database: ['Anglais', 'Espagnol', 'Sondage_LV2', 'General']


### Delete table

!!!! attention c'est définitif !!!!!

In [142]:
import sqlite3

Data = 'eleves/SQL/data.sqlite3'
def Delete_all_tables(filename):
    for i in list_tables(filename):
        conn = sqlite3.connect(filename)
        cursor = conn.cursor()
        if i != 'General':
            cursor.execute("DROP TABLE " + i + ";")
        tables = cursor.fetchall()
        conn.close()
    return

#Delete_all_tables(Data)

def Delete_one_table(filename, name_table):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("DROP TABLE " + name_table + ";")
    tables = cursor.fetchall()
    conn.close()
    return

#Delete_one_table(Data, "Allemand")

### create one general table 

entry: chemin de la database

In [61]:
import sqlite3

def Create_tables(filename):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE General (NAME VARCHAR(50),SURNAME VARCHAR(50),EMAIL VARCHAR(50),CLASS VARCHAR(5),LV1 VARCHAR(50),GRADE_LV1 INT,GROUP_LV1 INT,LV2 VARCHAR(50),GRADE_LV2 INT,GROUP_LV2 VARCHAR(50));")
    tables = cursor.fetchall()
    conn.close()
    return [table[0] for table in tables]

Data = 'eleves/SQL/data.sqlite3'
print("Tables in the database:", Create_tables(Data))

Tables in the database: []


### file general table 

In [ ]:
import sqlite3

def Create_tables(filename):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO General (NAME, colonne2, ..., colonneN) SELECT colonne_a_copier1, colonne_a_copier2, ..., colonne_a_copierN FROM Sondage_LV2;")
    tables = cursor.fetchall()
    conn.close()
    return [table[0] for table in tables]

Data = 'eleves/SQL/data.sqlite3'
#print("Tables in the database:", Create_tables(Data))

### add forecast

entry: nb of forecast, database, table(subject), mu, sigma

In [67]:
import sqlite3
import numpy as np

forecast = 30
Data = 'eleves/SQL/data.sqlite3'
table= 'Anglais'


mu_ANG = 6.68
sigma_ANG = 1.9
mu_ESP = 6.38
sigma_ESP = 1.69


def add_forcast(filename,table_add, nb_forecast, mu , sigma ):
    for i in range(0,nb_forecast):
        conn = sqlite3.connect(filename)
        cursor = conn.cursor()
        valeur_gaussienne = str(np.random.uniform(0, 10))
        cursor.execute("INSERT INTO " + table_add + " ('Nom', 'Prénom', 'mail', 'Note/10' ) VALUES ('forecast', 'forecast', 'forecast','" + valeur_gaussienne + "')")
        group = cursor.fetchall()
        conn.commit()
        conn.close()
    return group 

print(forecast,"forecast add", add_forcast(Data,table,forecast,mu_ANG, sigma_ANG ))


30 forecast add []


### Create groups

entry: database, table (subject), nb by class

In [114]:
import sqlite3

def extract_group(filename, table_add):
    conn = sqlite3.connect(filename)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM " + table_add + " ORDER BY [Note/10] DESC")
    group = cursor.fetchall()
    conn.close()
    return group


Data = 'eleves/SQL/data.sqlite3'
table= 'Anglais'
group_data = extract_group(Data, table)


def nomber_class(nomber_student, nomber_by_class):
    nomber_class = 0
    if (nomber_student % nomber_by_class) > ((nomber_student //nomber_by_class)*(2/3)) :
        nomber_class = nomber_student//nomber_by_class + 1
    else :
        nomber_class = nomber_student//nomber_by_class
    return nomber_class

nb_student = len(group_data)
nb_by_class = 16
nb_class = nomber_class(nb_student,nb_by_class)


def groups(list_student, number_class):
    
    # Step 2: Divide the sorted list of students into groups
    total_students = len(list_student)
    students_per_group = total_students // number_class
    groups = []
    start_idx = 0
    for i in range(number_class):
        end_idx = start_idx + students_per_group
        if i < total_students % number_class:  # If there are remaining students, distribute them
            end_idx += 1
        groups.append(list_student[start_idx:end_idx])
        start_idx = end_idx
    
    return groups

groups = groups(group_data, nb_class)
#print(groups)


#for group in groups:
    #print('\nGroupe\n', len(group), "\n", group)
    #mean = 0
    #for student in group:
        #mean =+ mean + float(student[3])
    #print(len(group))
    #print('Mean', mean/len(group))



### export the group in PDF 

entry : liste des groupes + matière du groupe

In [132]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER
import os


def pdf_maker(groups, langage):
    i = 1
    styles = getSampleStyleSheet()  

    for group in groups:
        pdf_path = f"eleves/pdf/{langage}_Group_{i}.pdf"
        if os.path.exists(pdf_path):
            os.remove(pdf_path)

        doc = SimpleDocTemplate(pdf_path, pagesize=letter)
        content = []

        title = Paragraph(f"<b>{langage}: Group {i}</b>", styles["Title"])
        content.append(title)
        content.append(Spacer(1, 12))  

        data = group
        table_data = [("Name", "Surname", "Email", "Mark")]
        table_data.extend(data)  

        table = Table(table_data)
        style = TableStyle([('BACKGROUND', (0,0), (-1,0), colors.grey),
                            ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
                            ('ALIGN', (0,0), (-1,-1), 'CENTER'),
                            ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
                            ('BOTTOMPADDING', (0,0), (-1,0), 12),
                            ('BACKGROUND', (0,1), (-1,-1), colors.beige),
                            ('GRID', (0,0), (-1,-1), 1, colors.black)])
        table.setStyle(style)

        content.append(table)

        content.append(Spacer(1, 12)) 
        centered_style = ParagraphStyle(name='Centered', parent=styles['Normal'], alignment=TA_CENTER)

        marks = [member[-1] for member in group]  # Supposer que la note est le dernier élément de chaque tuple
        mean_mark = sum(marks) / len(marks) if marks else 0

        summary = Paragraph(f"Number of student: {len(group)} ; Mark mean: {mean_mark:.2f}", centered_style)
        content.append(summary)
        doc.build(content)
        #print(f"Le fichier PDF '{pdf_path}' a été créé avec succès.")
        i += 1


pdf_maker(groups, "Anglais")
